In [1]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

import pandas as pd
import numpy as np
from datetime import datetime

2019-08-15 17:53:01,176 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-08-15 17:53:01,198 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-08-15 17:53:01,236 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-08-15 17:53:01,237 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-08-15 17:53:01,241 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-08-15 17:53:01,243 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-08-15 17:53:01,248 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-08-15 17:53:01,249 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-08-15 17:53:0

In [2]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "accredo_cancel_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [3]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

2019-08-15 17:53:01,825 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-15 17:53:01,852 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [4]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::accredo_cancel_before_active
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [5]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    # Column headers
    status_date: str = 'status_date'# Status date column
    ref_date: str = 'rec_date'# Referral date column
    patient: str = 'msa_patient_id' # Patient ID column
    pharm: str = 'pharm_code' # Pharmacy Name column
    product: str = 'medication' # Medication Name column
    status: str = 'status_code' # Status column
    substatus: str = 'sub_status' # Substatus column
    ic_status: str = 'integrichain_status' # IntegriChain Status column
    ic_substatus: str = 'integrichain_sub_status' # IntegriChain Substatus column
    pjh: str = 'Patient_Journey_Hierarchy' # Patient Journey Hierarchy column
    brand: str = 'brand' # Brand column (medication without strength)
        
    # Possible status values
    pending: str = 'PENDING' # Pending status 'PENDING'
    active: str = 'ACTIVE' # Active status 'ACTIVE'
    cancelled: str = 'CANCELLED' # Cancelled status 'CANCELLED'
    discontinued: str = 'DISCONTINUED' # Discontinued status 'DISCONTINUED'
        
    # Possible substatus values
    pending_new: str = 'NEW' # New substatus when status is 'PENDING'
    active_shipped: str = 'SHIPMENT'# Shipment substatus when status is 'ACTIVE'
    
    # Possible PJH values
    bvpa: str = 'BV/PA' # BV/PA pjh
    intake: str = 'Intake' # Intake pjh
    fulfillment: str = 'Fulfillment' # Fulfillment pjh
    transfer: str = 'Transferred' # Transferred pjh
    payer: str = 'Payer' # Payer pjh
    no_clarity: str = 'NO STATUS CLARITY' # Final result of enrichment
        
    # Pharmacy in question
    accredo: str = 'ACCREDO'

transform = Transform()

In [6]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

# Vars
patient = transform.patient
pharm = transform.pharm
product = transform.product
ref_date = transform.ref_date
status_date = transform.status_date
status = transform.status
substatus = transform.substatus
ic_status = transform.ic_status
ic_substatus = transform.ic_substatus
pjh = transform.pjh
brand = transform.brand

# Values
pending = transform.pending
active = transform.active
cancelled = transform.cancelled
discontinued = transform.discontinued
pending_new = transform.pending_new
active_shipped = transform.active_shipped
bvpa = transform.bvpa
intake = transform.intake
fulfillment = transform.fulfillment
transfer = transform.transfer
payer = transform.payer
no_clarity = transform.no_clarity
accredo = transform.accredo

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

### Transformation

In [7]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[3]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
final_dataframe = input_contract.fetch(filters=run_filter)

2019-08-15 17:53:02,097 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.


### Copy ingested data

In [8]:
df = final_dataframe.copy()

### Impute PJH and Preprocess Data

In [9]:
"""
This cell brings in the Patient Journey Hierarchy information from a local file.
This is just a placeholder to ensure data processes work as expected until the
true ingest data can be used.
"""

import os

pd.options.display.max_columns=999

os.chdir('{}'.format(os.path.expanduser('~')))
status_config = pd.read_csv('status_mapping.csv')

status_config = (
    status_config
    .assign(**{
        status: lambda x:(
            x['statusCode'].str.upper()
        ),
        substatus: lambda x:(
            x['subStatus'].str.upper()
        ),
        ic_status: lambda x:(
            x['integrichain_status'].str.upper()
        ),
        ic_substatus: lambda x:(
            x['integrichain_sub_status'].str.upper()
        ),
        pjh: lambda x:(
            x['Patient_Journey_Hierarchy'].str.upper()
        )
    })
    .drop(columns=[
        'statusCode',
        'subStatus'
    ])
)

df = (
    df
    .loc[:,
        [ref_date,
         patient,
         pharm,
         product,
         status_date,
         status,
         substatus]
        ]
    .fillna('NONE')
    .assign(**{
        brand: lambda x:(
            x[product].apply(lambda x: x.split()[0].strip())
        ),
        status: lambda x:(
            x[status].str.upper()
        ),
        substatus: lambda x:(
            x[substatus].str.upper()
        )
    })
    .assign(**{
        status_date: lambda x:(
            pd.to_datetime(
                x[status_date].str[:8].astype(str),
                errors='coerce'
        )),
        ref_date: lambda x:(
            pd.to_datetime(
                x[ref_date].str[:8].astype(str),
                errors='coerce'
        )),
        'min_status_date': lambda x:(
            x.groupby([patient, pharm, brand])[status_date].transform(min)
        )
    })
    .fillna(value={ref_date: 'min_status_date'})
    .drop(columns=['min_status_date'])
    .merge(status_config, how='left', on=[status, substatus])
    .drop_duplicates()
    .sort_values(
        [patient, pharm, brand, status_date]
    )
)

### Transformation

In [10]:
# Sort and reset index to join on later
df = (
    df
    .sort_values([patient, pharm, brand, status_date, ref_date])
    .reset_index(drop=False)
)

In [66]:
# Create secondary df to find the first active shipment date for
# each patient journey
min_shipped_df = (
    df
    .loc[
        (df[ic_status] == active) &
        (df[ic_substatus] == active_shipped)
    ]
    .groupby([patient, pharm, brand])
    [status_date]
    .min()
    .reset_index()
    .rename(columns={status_date: 'first_shipped_status_date'})
)

In [70]:
# Join to main df to get first active shipment date for every
# patient that has one
df = (
    df
    .merge(
        min_shipped_df,
        how='left'
    )
)

In [12]:
# Create a column that can be grouped on later to determine the
# first date where a desired care occurs
df = (
    df
    .assign(status_spree=(
    ~(
        (df[patient].eq(df[patient].shift(1))) &
        (df[pharm].eq(df[pharm].shift(1))) &
        (df[brand].eq(df[brand].shift(1))) &
        (df[ic_status].eq(df[ic_status].shift(1))) &
        (df[ic_substatus].eq(df[ic_substatus].shift(1)))
    )
    ).cumsum())
)

In [13]:
# Create column denoting the min status date of each spree
df = (
    df
    .assign(min_status_date=lambda x:(
        x.groupby('status_spree')[status_date].transform(min)
    ))
)

In [51]:
pd.options.display.max_rows=30

In [29]:
df[[ref_date, patient, pharm, brand, status_date, ic_status, ic_substatus, pjh, 'max_status_date', 'min_status_date', 'status_spree']]

,rec_date,msa_patient_id,pharm_code,ilumya,status_date,integrichain_status,integrichain_sub_status,Patient_Journey_Hierarchy,max_status_date,min_status_date,status_spree
0,2018-11-06 00:00:00,2120001,CVS,ILUMYA,2018-12-31,ACTIVE,SHIPMENT,FULFILLMENT,2018-12-31,2018-12-31,1
1,2018-12-20 00:00:00,2120001,CVS,ILUMYA,2018-12-31,ACTIVE,SHIPMENT,FULFILLMENT,2018-12-31,2018-12-31,1
2,2019-03-08 00:00:00,2120004,CVS,NONE,2019-03-08,PENDING,NEW,INTAKE,2019-03-14,2019-03-08,2
3,2019-03-11 00:00:00,2120004,CVS,NONE,2019-03-11,PENDING,APPEAL,BV/PA,2019-03-14,2019-03-11,3
4,2019-03-14 00:00:00,2120004,CVS,NONE,2019-03-14,CANCELLED,INSURANCE DENIED,PAYER,2019-03-14,2019-03-14,4
5,2019-03-25 00:00:00,2120006,CVS,ILUMYA,2019-03-25,PENDING,PATIENT CONTACT,FULFILLMENT,2019-04-05,2019-03-25,5
6,2019-04-05 00:00:00,2120006,CVS,ILUMYA,2019-04-05,CANCELLED,PATIENT RESPONSE,PATIENT,2019-04-05,2019-04-05,6
7,2019-03-15 00:00:00,2120006,CVS,NONE,2019-03-15,PENDING,APPEAL,BV/PA,2019-03-15,2019-03-15,7
8,2019-01-04 00:00:00,2120009,BRV,ILUMYA,2019-01-03,PENDING,OTHER,FULFILLMENT,2019-03-05,2019-01-03,8
9,2019-01-05 00:00:00,2120009,BRV,ILUMYA,2019-01-04,PENDING,BENEFITS,BV/PA,2019-03-05,2019-01-04,9


In [24]:
(
    df
    .sort_values([patient, pharm, brand, status_date, ref_date])
    .assign(prev_status=lambda x:(
        x[ic_status].shift(1)
    ))
).head(10)

,index,rec_date,msa_patient_id,pharm_code,medication,status_date,status_code,sub_status,ilumya,integrichain_sub_status,integrichain_status,Patient_Journey_Hierarchy,max_status_date,status_spree,min_status_date,last_status,last_substatus,day_difference,prev_status,prev_substatus,prev_check
0,2229,2018-11-06 00:00:00,2120001,CVS,ILUMYA SD PFS,2018-12-31,ACTIVE,SHIPMENT,ILUMYA,SHIPMENT,ACTIVE,FULFILLMENT,2018-12-31,1,2018-12-31,NaN,NaN,NaT,NaN,NaN,False
1,2230,2018-12-20 00:00:00,2120001,CVS,ILUMYA SD PFS,2018-12-31,ACTIVE,SHIPMENT,ILUMYA,SHIPMENT,ACTIVE,FULFILLMENT,2018-12-31,1,2018-12-31,ACTIVE,SHIPMENT,0 days,ACTIVE,SHIPMENT,False
2,2887,2019-03-08 00:00:00,2120004,CVS,NONE,2019-03-08,PENDING,NEW,NONE,NEW,PENDING,INTAKE,2019-03-14,2,2019-03-08,NaN,NaN,NaT,NaN,NaN,False
3,2915,2019-03-11 00:00:00,2120004,CVS,NONE,2019-03-11,PENDING,APPEAL,NONE,APPEAL,PENDING,BV/PA,2019-03-14,3,2019-03-11,NaN,NaN,NaT,NaN,NaN,False
4,2996,2019-03-14 00:00:00,2120004,CVS,NONE,2019-03-14,CANCELLED,INSURANCE DENIED,NONE,INSURANCE DENIED,CANCELLED,PAYER,2019-03-14,4,2019-03-14,CANCELLED,INSURANCE DENIED,0 days,PENDING,APPEAL,False
5,3163,2019-03-25 00:00:00,2120006,CVS,ILUMYA SD PFS,2019-03-25,PENDING,PATIENT CONTACT,ILUMYA,PATIENT CONTACT,PENDING,FULFILLMENT,2019-04-05,5,2019-03-25,NaN,NaN,NaT,NaN,NaN,False
6,3461,2019-04-05 00:00:00,2120006,CVS,ILUMYA SD PFS,2019-04-05,CANCELLED,PATIENT RESPONSE,ILUMYA,PATIENT RESPONSE,CANCELLED,PATIENT,2019-04-05,6,2019-04-05,CANCELLED,PATIENT RESPONSE,0 days,PENDING,PATIENT CONTACT,False
7,3030,2019-03-15 00:00:00,2120006,CVS,NONE,2019-03-15,PENDING,APPEAL,NONE,APPEAL,PENDING,BV/PA,2019-03-15,7,2019-03-15,PENDING,APPEAL,0 days,CANCELLED,PATIENT RESPONSE,True
8,8747,2019-01-04 00:00:00,2120009,BRV,ILUMYA 100MG/ML PFS INJ,2019-01-03,PENDING,OTHER,ILUMYA,OTHER,PENDING,FULFILLMENT,2019-03-05,8,2019-01-03,NaN,NaN,NaT,NaN,NaN,False
9,8767,2019-01-05 00:00:00,2120009,BRV,ILUMYA 100MG/ML PFS INJ,2019-01-04,PENDING,BENEFITS,ILUMYA,BENEFITS,PENDING,BV/PA,2019-03-05,9,2019-01-04,NaN,NaN,NaT,NaN,NaN,False


In [14]:
# Drop records from df to find last status and substatus
last_status_df = (
    df
    .sort_values([patient, pharm, brand, status_date, ref_date])
    # Also create previous status and substatus columns
    .assign(prev_status=lambda x:(
        x[ic_status].shift(1)
    ))
    .assign(prev_substatus=lambda x:(
        x[ic_substatus].shift(1)
    ))
    # Find the previous status date and num of days until last status
    .assign(prev_status_date=lambda x:(
        x[status_date].shift(1)
    ))
    .assign(day_difference=lambda x:(
        x['max_status_date'] - x['min_status_date']
    ))
    # Drop duplicates so we only get the last status for each patient journey
    .drop_duplicates(subset=[patient, pharm, brand], keep='last')
    [[patient, pharm, brand, status_date, ref_date, ic_status, ic_substatus, 'day_difference', 'prev_status', 'prev_substatus']]
    .rename(columns={ic_status: 'last_status', ic_substatus: 'last_substatus'})
    .drop(columns=[patient, pharm, brand, status_date, ref_date])
)

In [15]:
# Merge to df to obtain the desired columns
df = (
    df
    .join(last_status_df, how='left')
)

In [16]:
# Create column denoting prev status and substatus that are in one of the three cases
df.loc[:, 'prev_check'] = (
    (
        (df.prev_status == pending) &
        (df.prev_substatus == pending_new)
    ) |
    df.prev_status.isin([cancelled, discontinued])
)

In [17]:
# Apply enrichment using loc with the case structure
df.loc[(
    #(df[pharm] == accredo) &
    (df['prev_check'] == True) &
    (
        (df['last_status'] == active) &
        (df['last_substatus'] == active_shipped)
    ) &
    (df['day_difference'] > np.timedelta64(2, 'D'))
), pjh] = no_clarity

# IMPACT

In [18]:
len(df[df[pjh] == no_clarity]) / len(df)

0.0

In [19]:
len(df)

10033

In [20]:
len(df[df[pjh] == no_clarity])

0

### Format

In [19]:
final_dataframe = (
    df
    .set_index('index')
    .sort_index()
    .drop(columns=['max_status_date', 'last_status', 'last_substatus', 'day_difference', 'prev_status', 'prev_substatus', 'prev_check'])
)

In [20]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

### Publish

In [21]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()

2019-08-15 13:56:03,997 - core.dataset_contract.DatasetContract - INFO - Publishing dataframe to s3 location s3://ichain-dev/ds301_accredo_enrichment/sun/ilumya/enrich/accredo_cancel_before_active with run ID 594643.
2019-08-15 13:56:04,010 - core.dataset_contract.DatasetContract - DEBUG - Publishing dataframe to Redshift Spectrum database ichain_core to schema.table                 data_core.sun_ilumya_accredo_cancel_before_active...
2019-08-15 13:56:04,379 - s3parq.publish_redshift - INFO - Running query to create schema: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';


/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ProgrammingError: (psycopg2.ProgrammingError) permission denied for database ichain_core

[SQL: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';]
(Background on this error at: http://sqlalche.me/e/f405)